In [4]:
import os, sys, glob
from tqdm import tqdm
import avstack
import avapi
import cv2
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

data_dir = '../data/MOT15/'

MSM = avapi.mot15.MOT15SceneManager(data_dir=data_dir)
MSD = MSM.get_scene_dataset_by_name('ADL-Rundle-6')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Run Algorithms

In [2]:
framerate = 30.0
detector = avstack.modules.perception.object2dfv.MMDetObjectDetector2D(
    model='fasterrcnn', dataset='coco-person', deploy=False)
tracker = avstack.modules.tracking.tracker2d.SortTracker2D()

Loads checkpoint by local backend from path: /home/spencer/Documents/Projects/AVstack/platforms/demo-platform/third_party/lib-avstack-core/third_party/mmdetection/checkpoints/coco-person/faster_rcnn_r50_fpn_1x_coco-person_20201216_175929-d022e227.pth


In [3]:
all_dets = {}
all_tracks = {}
i_frame_start = 390
n_frames = 30; n_frames = min(len(MSD), n_frames)
cvt_to_rgb = True
ego_ref = MSD.get_ego_reference(frame=1)

all_frames = []
for i_frame, frame in tqdm(enumerate(MSD.frames[i_frame_start:i_frame_start+n_frames]), total=n_frames):
    all_frames.append(frame)
    # -- load image in avstack standard format
    img = MSD.get_image(frame=frame)
    
    # -- run detections and tracking
    dets = detector(img)
    print(dets)
    tracks = tracker(detections=dets, frame=img.frame, t=img.timestamp, platform=ego_ref)
    
    # -- save results
    all_dets[frame] = dets
    all_tracks[frame] = tracks

  7%|███▊                                                     | 2/30 [00:00<00:10,  2.57it/s]

0 elements at frame 391, time 13.0, ID: 
[]
0 elements at frame 392, time 13.033333333333333, ID: 
[]


 13%|███████▌                                                 | 4/30 [00:01<00:06,  4.04it/s]

0 elements at frame 393, time 13.066666666666666, ID: 
[]
0 elements at frame 394, time 13.1, ID: 
[]


 20%|███████████▍                                             | 6/30 [00:01<00:04,  4.84it/s]

0 elements at frame 395, time 13.133333333333333, ID: 
[]
0 elements at frame 396, time 13.166666666666666, ID: 
[]


 23%|█████████████▎                                           | 7/30 [00:01<00:05,  3.92it/s]

0 elements at frame 397, time 13.2, ID: 
[]


KeyboardInterrupt: 

In [ ]:
idx_frame = 10
frame = all_frames[idx_frame]
img = MSD.get_image(frame=frame)
objs = MSD.get_objects(frame=frame)
avapi.visualize.snapshot.show_image_with_boxes(img, objs, inline=True) 
avapi.visualize.snapshot.show_image_with_boxes(img, all_dets[frame], inline=True)
avapi.visualize.snapshot.show_image_with_boxes(img, all_tracks[frame], inline=True) 